# Project Proposal

This project is an implementation of a reinforcement learning method to find a safe route from two points in New York City. We use crime map of New York City to help us to build the map in our code. We also try to make a tool that can help you to build your map with a real map.

# Implementation

We use Q-Learning as our internal learning algorithm. Q-learning is a model-free reinforcement learning algorithm. The goal of Q-learning is to learn a policy, which tells an agent what action to take under what circumstances. It does not require a model (hence the connotation "model-free ") of the environment, and it can handle problems with stochastic transitions and rewards, without requiring adaptations. For any finite Markov decision process (FMDP), Q-learning finds a policy that is optimal in the sense that it maximizes the expected value of the total reward over any and all successive steps, starting from the current state. Q-learning can identify an optimal action-selection policy for any given FMDP, given infinite exploration time and a partly-random policy. "Q" names the function that returns the reward used to provide the reinforcement and can be said to stand for the "quality" of an action taken in a given state. (from Wikipedia)

# Running Instruction

* Run the code below and you will see a gui window pops up.
* Marked the map using the tools we have. (Note that you have to at least set the start and end position)
* Click "Start Learning" and see the learning process of the agent.

### ***The GUI window may not fully display on laptops with 13' screen, so we recommand using big screen(more than 21') to make sure its works.*** 
### For detailed demo, please check out YouTube video

In [1]:
import os
import random
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import *
from enum import Enum

MOVE_LEFT = 'left'
MOVE_RIGHT = 'right'
MOVE_UP = 'up'
MOVE_DOWN = 'down'

In [2]:
class QLAgent:
    def __init__(self, epsilon, learning_rate, discount_factor, actions):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.actions = actions
        self.Q_table = pd.DataFrame(columns=actions)

    def learn(self, cur_state, action, reward, next_state):
        self.create_state(next_state)
        q_predict = self.Q_table.loc[cur_state, action]
        if self.not_finished(reward):
            q_reality = reward + self.discount_factor * self.Q_table.loc[next_state, :].max()
        else:
            q_reality = reward
        self.Q_table.loc[cur_state, action] += self.learning_rate * (q_reality - q_predict)

    def create_state(self, state):
        if state not in self.Q_table.index:
            self.Q_table = self.Q_table.append(
                pd.Series(
                    data=[0] * len(self.actions),
                    index=self.actions,
                    name=state
                )
            )

    def init_index(self, max_row, max_col):
        index = []
        for row in range(max_row):
            for col in range(max_col):
                index.append(Point(row, col).toString())
        return index

    def choose_action(self, state):
        self.create_state(state)
        temp = np.random.uniform()

        if temp < self.epsilon:
            actions = self.Q_table.loc[state, :]
            action = self.choose_best_action(actions)
        elif temp >= self.epsilon:
            action = np.random.choice(self.actions)

        return action

    def not_finished(self, reward):
        return reward < 800 and reward > -800

    def choose_best_action(self, actions):
        q_table_cols = self.Q_table.columns
        max_action_value = -sys.maxsize
        max_action_value_list = []

        for i in range(len(q_table_cols)):
            action_value = actions[i]
            q_tabl_col = q_table_cols[i]

            if action_value > max_action_value:
                max_action_value = action_value
                max_action_value_list = [q_tabl_col]
            elif action_value == max_action_value:
                max_action_value_list.append(q_tabl_col)
            else:
                continue

        if len(max_action_value_list) > 1:
            random_action_index = random.randint(0, len(max_action_value_list) - 1)
            best_action = max_action_value_list[random_action_index]
        else:
            best_action = max_action_value_list[0]

        return best_action



In [3]:
class MapEnv:
    def __init__(self, max_row, max_col, traveler, destination, crime_high, crime_low, crime_mid, crime_extreme, refresh_interval, obstacles=None, ):
        self.max_row = max_row
        self.max_col = max_col
        self.init_worker = traveler.clone()
        self.traveler = traveler
        self.destination = destination
        self.crime_high = crime_high
        self.crime_mid = crime_mid
        self.crime_low = crime_low
        self.crime_extreme = crime_extreme
        self.refresh_interval = refresh_interval
        self.obstacles = obstacles

    def move(self, action):
        if action == MOVE_LEFT:
            if self.traveler.col > 0:
                self.traveler.col -= 1
        elif action == MOVE_RIGHT:
            if self.traveler.col < self.max_col - 1:
                self.traveler.col += 1
        elif action == MOVE_UP:
            if self.traveler.row > 0:
                self.traveler.row -= 1
        elif action == MOVE_DOWN:
            if self.traveler.row < self.max_row - 1:
                self.traveler.row += 1
        else:
            raise Exception('Not supported action: {}'.format(action))

        return self.feedback()

    def feedback(self):
        state = self.traveler.toString()
        reward = 0
        if self.traveler.equal(self.destination):
            reward = 1000
        for low in self.crime_low:
            if self.traveler.equal(low):
                reward -= 30
        for mid in self.crime_mid:
            if self.traveler.equal(mid):
                reward -= 100
        for high in self.crime_high:
            if self.traveler.equal(high):
                reward -= 400
        for extreme in self.crime_extreme:
            if self.traveler.equal(extreme):
                reward -= 800
        for obstacle in self.obstacles:
            if self.traveler.equal(obstacle):
                reward = -1000
        return state, reward

    def reset(self):
        self.traveler = self.init_worker.clone()
        return self.traveler.toString()

    def display(self):
        os.system('clear')
        arr = np.zeros((self.max_row, self.max_col))
        arr[self.destination.row][self.destination.col] = 8
        arr[self.traveler.row][self.traveler.col] = 1
        for obstacle in self.obstacles:
            arr[obstacle.row][obstacle.col] = 7
        for low in self.crime_low:
            arr[low.row][low.col] = -1
        for mid in self.crime_mid:
            arr[mid.row][mid.col] = -2
        for high in self.crime_high:
            arr[high.row][high.col] = -3
        for extreme in self.crime_extreme:
            arr[extreme.row][extreme.col] = -4
        print(arr)


class Point:
    def __init__(self, row, col):
        self.row = row
        self.col = col

    def equal(self, other_point):
        return self.row == other_point.row and self.col == other_point.col

    def __eq__(self, other):
        return self.row == other.row and self.col == other.col

    def toString(self):
        return "({},{})".format(str(self.row), str(self.col))

    def clone(self):
        return Point(self.row, self.col)



In [4]:
class Operation(Enum):
    OBSTACLE = 1
    DANGER_1 = 2
    DANGER_2 = 3
    DANGER_3 = 4
    DANGER_4 = 5
    ERASER = 6
    START = 7
    END = 8

In [ ]:
if __name__ == '__main__':
    # Global Variables
    MAX_ROW = 19
    MAX_COL = 7
    # Traveler/ destination/ obstacles setup
    traveler = Point(-1, -1)
    destination = Point(-1, -1)
    obstacles = []
    crime_low = []
    crime_mid = []
    crime_high = []
    crime_extreme = []
    startLearning = False
    # =================== GUI Mark Tool ====================
    # Create a grid of None to store the references to the tiles
    tiles = [[None for _ in range(MAX_COL)] for _ in range(MAX_ROW)]
    marker = [[None for _ in range(MAX_COL)] for _ in range(MAX_ROW)]
    currentMode = Operation.OBSTACLE
    # Create the window, a canvas and the mouse click event binding
    root = tk.Tk()
    # Load map
    img = PhotoImage(file="nyc_crime.png")
    c = tk.Canvas(root, width=img.width(), height=img.height(), borderwidth=5, background='white')
    c.create_image(0, 0, anchor=NW, image=img)
    # Add scroll bar
    # scrollbar = Scrollbar(root)
    # scrollbar.config(command=c.yview)
    # scrollbar.pack(side=RIGHT, fill=Y)
    # Add widgets aka buttons
    f = tk.Frame(c)  # Create the frame which will hold the widgets
    c.pack(side="left", fill="both", expand=True)
    c.create_window(0, 0, window=f, anchor='nw')

    col_width = -1
    row_height = -1
    # Label of status
    statusLabel = StringVar()
    Label(f, textvariable=statusLabel).pack()
    succeedLabel = StringVar()
    Label(f, textvariable=succeedLabel).pack()
    failedLabel = StringVar()
    Label(f, textvariable=failedLabel).pack()

    statusLabel.set("Not Started")
    succeedLabel.set("Succeed Steps: ")
    failedLabel.set("Failed Steps: ")
    # root.update()
    # c.config(scrollregion=c.bbox("all"))


    def mark(event):
        global traveler
        global destination
        global col_width
        global row_height
        col_width = c.winfo_width() / MAX_COL
        row_height = c.winfo_height() / MAX_ROW
        # Calculate column and row number
        col = int(event.x // col_width)
        row = int(event.y // row_height)
        # If the tile is not filled, create a rectangle
        if not tiles[row][col] and currentMode == Operation.OBSTACLE:
            marker[row][col] = c.create_rectangle(col * col_width, row * row_height, (col + 1) * col_width,
                                                  (row + 1) * row_height, fill="black")
            tiles[row][col] = -1000
            obstacles.append(Point(row, col))
        if not tiles[row][col] and currentMode == Operation.DANGER_1:
            marker[row][col] = c.create_rectangle(col * col_width, row * row_height, (col + 1) * col_width,
                                                  (row + 1) * row_height, fill="blue")
            tiles[row][col] = -30
            crime_low.append(Point(row, col))
        if not tiles[row][col] and currentMode == Operation.DANGER_2:
            marker[row][col] = c.create_rectangle(col * col_width, row * row_height, (col + 1) * col_width,
                                                  (row + 1) * row_height, fill="yellow")
            tiles[row][col] = -100
            crime_mid.append(Point(row, col))
        if not tiles[row][col] and currentMode == Operation.DANGER_3:
            marker[row][col] = c.create_rectangle(col * col_width, row * row_height, (col + 1) * col_width,
                                                  (row + 1) * row_height, fill="orange")
            tiles[row][col] = -400
            crime_high.append(Point(row, col))
        if not tiles[row][col] and currentMode == Operation.DANGER_4:
            marker[row][col] = c.create_rectangle(col * col_width, row * row_height, (col + 1) * col_width,
                                                  (row + 1) * row_height, fill="red")
            tiles[row][col] = -800
            crime_extreme.append(Point(row, col))
        if not tiles[row][col] and currentMode == Operation.START:
            if traveler.row == -1 and traveler.col == -1:
                traveler = Point(row, col)
                marker[row][col] = c.create_rectangle(col * col_width, row * row_height, (col + 1) * col_width,
                                                      (row + 1) * row_height, fill="green")
                tiles[row][col] = 1

        if not tiles[row][col] and currentMode == Operation.END:
            if destination.row == -1 and destination.col == -1:
                destination = Point(row, col)
                marker[row][col] = c.create_rectangle(col * col_width, row * row_height, (col + 1) * col_width,
                                                      (row + 1) * row_height, fill="green")
                tiles[row][col] = 1000
        if tiles[row][col] and currentMode == Operation.ERASER:
            if traveler.row == row and traveler.col == col:
                traveler = Point(-1, -1)
            if destination.row == row and destination.col == col:
                destination = Point(-1, -1)
            c.delete(marker[row][col])
            marker[row][col] = None
            tiles[row][col] = None
            if Point(row, col) in obstacles:
                obstacles.remove(Point(row, col))
            if Point(row, col) in crime_low:
                crime_low.remove(Point(row, col))
            if Point(row, col) in crime_mid:
                crime_mid.remove(Point(row, col))
            if Point(row, col) in crime_high:
                crime_high.remove(Point(row, col))
            if Point(row, col) in crime_extreme:
                crime_extreme.remove(Point(row, col))
        c.update()


    c.bind("<Button-1>", mark)
    c.bind("<B1-Motion>", mark)


    def obstacleBtnHandler(event):
        global currentMode
        currentMode = Operation.OBSTACLE


    def danger1BtnHandler(event):
        global currentMode
        currentMode = Operation.DANGER_1


    def danger2BtnHandler(event):
        global currentMode
        currentMode = Operation.DANGER_2


    def danger3BtnHandler(event):
        global currentMode
        currentMode = Operation.DANGER_3


    def danger4BtnHandler(event):
        global currentMode
        currentMode = Operation.DANGER_4


    def eraserBtnHandler(event):
        global currentMode
        currentMode = Operation.ERASER


    def startBtnHandler(event):
        global currentMode
        currentMode = Operation.START


    def endBtnHandler(event):
        global currentMode
        currentMode = Operation.END


    def startLearningBtnHandler(event):
        refresh_interval = 0.01
        # Number of iterations
        episode = 200
        # QLearning settings
        epsilon = 0.9
        learning_rate = 0.05
        discount_factor = 0.9
        # Move choices
        actions = [MOVE_LEFT, MOVE_RIGHT, MOVE_UP, MOVE_DOWN]
        # Init map environment using the variables above
        env = MapEnv(
            max_row=MAX_ROW,
            max_col=MAX_COL,
            traveler=traveler,
            destination=destination,
            obstacles=obstacles,
            refresh_interval=refresh_interval,
            crime_low=crime_low,
            crime_mid=crime_mid,
            crime_high=crime_high,
            crime_extreme=crime_extreme,
        )
        # Init learning agent
        agent = QLAgent(
            epsilon=epsilon,
            learning_rate=learning_rate,
            discount_factor=discount_factor,
            actions=actions
        )
        successful_routines = []
        failed_routines = []
        # Show the map
        env.display()

        for eps in range(1, episode + 1):
            # Start from init location
            cur_state = env.reset()
            step_counter = 0

            while True:
                step_counter += 1
                statusLabel.set("Episode: " + str(eps) + ", Step: " + str(step_counter))
                succeedText = str(successful_routines).strip('[]').split(',')
                failedText = str(failed_routines).strip('[]').split(',')
                succeedLabel.set("Succeed Routines: " + str(succeedText[len(succeedText) - 5 if len(succeedText) > 5 else 0:len(succeedText)]))
                failedLabel.set("Failed Routines: " + str(failedText[len(failedText) - 5 if len(failedText) > 5 else 0:len(failedText)]))

                env.display()

                if (Point(env.traveler.row, env.traveler.col) not in obstacles) and (
                        Point(env.traveler.row, env.traveler.col) not in crime_low) and (
                        Point(env.traveler.row, env.traveler.col) not in crime_mid) and (
                        Point(env.traveler.row, env.traveler.col) not in crime_high) and (
                        Point(env.traveler.row, env.traveler.col) not in crime_extreme):
                    c.delete(marker[env.traveler.row][env.traveler.col])
                    marker[env.traveler.row][env.traveler.col] = None
                action = agent.choose_action(cur_state)

                next_state, reward = env.move(action)
                if (Point(env.traveler.row, env.traveler.col) not in obstacles) and (
                        Point(env.traveler.row, env.traveler.col) not in crime_low) and (
                        Point(env.traveler.row, env.traveler.col) not in crime_mid) and (
                        Point(env.traveler.row, env.traveler.col) not in crime_high) and (
                        Point(env.traveler.row, env.traveler.col) not in crime_extreme):
                    marker[env.traveler.row][env.traveler.col] = c.create_rectangle(env.traveler.col * col_width,
                                                                                    env.traveler.row * row_height,
                                                                                    (env.traveler.col + 1) * col_width,
                                                                                    (env.traveler.row + 1) * row_height,
                                                                                    fill="green")
                c.update()

                agent.learn(
                    cur_state=cur_state,
                    action=action,
                    reward=reward,
                    next_state=next_state
                )

                cur_state = next_state

                if reward > 800 or reward < -800:
                    # if reward != 0:
                    break

            if reward > 800:
                successful_routines.append(step_counter)
            elif reward < -800:
                failed_routines.append(step_counter)

            print(
                'total episode: {}\n'
                'current episode: {}\n'
                'reward: {}\nsteps: {}\n'
                'successful steps record: {}\n'
                'failed steps record: {}'
                    .format(
                    episode,
                    eps,
                    reward,
                    step_counter,
                    successful_routines,
                    failed_routines
                )
            )


    obstacleBtn = Button(root, text="Obstacle")
    danger1Btn = Button(root, text="Danger Zone 1")
    danger2Btn = Button(root, text="Danger Zone 2")
    danger3Btn = Button(root, text="Danger Zone 3")
    danger4Btn = Button(root, text="Danger Zone 4")
    eraserBtn = Button(root, text="Eraser")
    startBtn = Button(root, text="Start")
    endBtn = Button(root, text="End")
    startLearningBtn = Button(root, text="Start Learning", width=20, height=10)
    obstacleBtn.pack()
    danger1Btn.pack()
    danger2Btn.pack()
    danger3Btn.pack()
    danger4Btn.pack()
    eraserBtn.pack()
    startBtn.pack()
    endBtn.pack()
    startLearningBtn.pack()
    obstacleBtn.bind("<Button-1>", obstacleBtnHandler)
    danger1Btn.bind("<Button-1>", danger1BtnHandler)
    danger2Btn.bind("<Button-1>", danger2BtnHandler)
    danger3Btn.bind("<Button-1>", danger3BtnHandler)
    danger4Btn.bind("<Button-1>", danger4BtnHandler)
    eraserBtn.bind("<Button-1>", eraserBtnHandler)
    startBtn.bind("<Button-1>", startBtnHandler)
    endBtn.bind("<Button-1>", endBtnHandler)
    startLearningBtn.bind("<Button-1>", startLearningBtnHandler)

    # =================== GUI Mark Tool ====================
    root.mainloop()

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 8. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 8. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/charlesdai/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-6-be24ff90aa03>", line 222, in startLearningBtnHandler
    action = agent.choose_action(cur_state)
  File "<ipython-input-2-a6d351f178ce>", line 41, in choose_action
    action = self.choose_best_action(actions)
  File "<ipython-input-2-a6d351f178ce>", line 68, in choose_best_action
    random_action_index = random.randint(0, len(max_action_value_list) - 1)
NameError: name 'random' is not defined


### You can also manully mark the map like the example below

In [9]:
if __name__ == '__main__':
    refresh_interval = 0.05

    episode = 150

    epsilon = 0.9
    learning_rate = 0.05
    discount_factor = 0.9  #

    max_row = 19
    max_col = 7
    actions = [MOVE_LEFT, MOVE_RIGHT, MOVE_UP, MOVE_DOWN]
    worker = Point(1, 6)
    treasure = Point(2, 3)
    crime_low = [
        Point(0, 1),
        Point(0, 6),
        Point(2, 1),
        Point(2, 6),
        Point(3, 4),
        Point(4, 6),
        Point(6, 1),
        Point(6, 3),
        Point(6, 5),
        Point(8, 3),
        Point(8, 5),
        Point(9, 4),
        Point(9, 6),
        Point(10, 5),
        Point(11, 4),
        Point(13, 0),
        Point(13, 6),
        Point(14, 1),
        Point(14, 3),
        Point(15, 0),
        Point(16, 1),
        Point(16, 4),
        Point(17, 4),
        Point(18, 1),
    ]

    crime_mid = [
        Point(0, 3),
        Point(4, 5),
        Point(10, 3),
        Point(13, 2),
        Point(17, 6),
        Point(18, 2),
        Point(18, 5),

    ]

    crime_high = [
        Point(8, 1),
        Point(18, 0),

    ]

    crime_extreme = [
        Point(5, 6),


    ]



    obstacles = [
        Point(1, 1),
        Point(1, 3),
        Point(1, 5),
        Point(3, 1),
        Point(3, 3),
        Point(3, 5),
        Point(5, 1),
        Point(5, 3),
        Point(5, 5),
        Point(7, 1),
        Point(7, 3),
        Point(7, 5),
        Point(9, 1),
        Point(9, 3),
        Point(9, 5),
        Point(11, 1),
        Point(11, 3),
        Point(11, 5),
        Point(13, 1),
        Point(13, 3),
        Point(13, 5),
        Point(15, 1),
        Point(15, 3),
        Point(15, 5),
        Point(17, 1),
        Point(17, 3),
        Point(17, 5),


    ]

    env = MapEnv(
        max_row=max_row,
        max_col=max_col,
        traveler=worker,
        destination=treasure,
        obstacles=obstacles,
        crime_extreme=crime_extreme,
        crime_low=crime_low,
        crime_mid=crime_mid,
        crime_high=crime_high,
        refresh_interval=refresh_interval
    )
    agent = QLAgent(
        epsilon=epsilon,
        learning_rate=learning_rate,
        discount_factor=discount_factor,
        actions=actions
    )
    successful_step_counter_arr = []
    failed_step_counter_arr = []

    env.display()

    for eps in range(1, episode + 1):

        cur_state = env.reset()
        step_counter = 0

        while True:
            step_counter += 1

            env.display()

            action = agent.choose_action(cur_state)

            next_state, reward = env.move(action)

            agent.learn(
                cur_state=cur_state,
                action=action,
                reward=reward,
                next_state=next_state
            )

            cur_state = next_state

            if reward > 800 or reward < -800:

            # if reward != 0:
                break

        if reward > 800:
            successful_step_counter_arr.append(step_counter)
        elif reward < -800:
            failed_step_counter_arr.append(step_counter)

        print(
            'total episode: {}\n'
            'current episode: {}\n'
            'reward: {}\nsteps: {}\n'
            'successful steps record: {}\n'
            'failed steps record: {}'
                .format(
                episode,
                eps,
                reward,
                step_counter,
                successful_step_counter_arr,
                failed_step_counter_arr
            )
        )


        # print(agent.q_table)
        # print('successful steps record: {}'.format(succeed_step_counter_arr))

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  1.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  1.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  1.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
(1,6)
[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  1.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  1.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
(3,6)
[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  1.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
(1,6)
[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  1.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  1.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
(1,6)
[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  1.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 

...

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  1.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
(1,6)
[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  0.  1. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
(2,5)
total episode: 150
current episode: 129
reward: -1000
steps: 3
successful steps record: [56, 15, 62, 29, 10, 49, 17, 45, 17, 32, 7, 62, 7, 18, 29, 28, 16, 11, 7, 16, 14, 55, 15, 4, 6, 10, 19, 18, 13, 29, 16, 58, 14, 9, 8, 42, 8, 4, 4, 4, 5, 4, 4, 6, 4, 4, 4, 4, 4, 6, 4, 8, 4, 4, 4, 4, 9, 4, 4, 4, 4, 4, 15, 4, 4]
failed steps record: [8, 1, 7, 37, 40, 10, 19, 25, 1, 23, 37, 45, 20, 11, 18, 11, 57, 33, 

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  0.  1. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
(2,5)
[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  1.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  0.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
(2,6)
[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  0.  1. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 

[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  0.  1. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  7. -2.  7.  0.  7. -1.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [-1.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  0. -1.  0.  0.]
 [ 0.  7.  0.  7. -1.  7. -2.]
 [-3. -1. -2.  0.  0. -2.  0.]]
(2,5)
[[ 0. -1.  0. -2.  0.  0. -1.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -1.  0.  8.  1.  0. -1.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0. -2. -1.]
 [ 0.  7.  0.  7.  0.  7. -4.]
 [ 0. -1.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7.  0.  7.  0.]
 [ 0. -3.  0. -1.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7. -1.]
 [ 0.  0.  0. -2.  0. -1.  0.]
 [ 0.  7.  0.  7. -1.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 